##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Composição de algoritmos de aprendizado

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Antes de começar

Antes de começar, execute o código abaixo para que o ambiente seja configurado corretamente. Se não for exibida uma saudação, consulte as instruções de [instalação](../install.md). 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
from collections.abc import Callable

import tensorflow as tf
import tensorflow_federated as tff

**OBSERVAÇÃO**: foi verificado que este Colab funciona com a [versão mais recente lançada](https://github.com/tensorflow/federated#compatibility) do pacote pip `tensorflow_federated`, mas o projeto TensorFlow Federated ainda está em desenvolvimento pré-lançamento e pode não funcionar no `main`.

# Composição de algoritmos de aprendizado

O [tutorial "Crie seu próprio algoritmo de aprendizado federado"](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) usou o federated core do TFF para implementar diretamente uma versão do algoritmo de cálculo federado de médias (FedAvg).

Neste tutorial, você usará os componentes do aprendizado federado na API do TFF para criar algoritmos de aprendizado federado de maneira modular, sem precisar implementar tudo novamente do zero.

Para os fins deste tutorial, você implementará uma variante da FedAvg que usa o recorte de gradiente através do treinamento local.

## Blocos básicos do algoritmo de aprendizado

Em um alto nível, vários algoritmos de aprendizado podem ser divididos em 4 componentes separados, chamados de **blocos básicos**. São eles:

1. Distribuidor (ou seja, comunicação do servidor para o cliente)
2. Trabalho do cliente (ou seja, computação do cliente local)
3. Agregador (ou seja, comunicação do cliente para o servidor)
4. Finalizador (ou seja, computação do servidor que usa saídas agregadas do cliente)

Enquanto o [tutorial "Crie seu próprio algoritmo de aprendizado federado"](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) implementou todos esses blocos básicos do zero, isso é geralmente desnecessário. Em vez disso, você pode reutilizar blocos básicos de algoritmos semelhantes.

Nesse caso, para implementar a FedAvg com o recorte de gradiente, você só precisa modificar o bloco básico de **trabalho do cliente**. Os outros blocos podem ser idênticos aos usados na FedAvg comum.

# Implemente o trabalho do cliente

Primeiro, vamos escrever a lógica do TF que realiza o treinamento do modelo local com o recorte de gradiente. Para simplificar, os gradientes recortados terão uma norma de no máximo 1.

## Lógica do TF

In [ ]:
@tf.function
def client_update(model: tff.learning.models.VariableModel,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.models.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.models.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

Há alguns pontos importantes no código acima. Primeiro, ele monitora o número de exemplos vistos, já que isso constituirá o *peso* da atualização do cliente (ao calcular a média dos clientes).

Segundo, ele usa [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult) para empacotar a saída. Esse tipo de retorno é utilizado para padronizar os blocos básicos de trabalho do cliente em `tff.learning`.

## Crie um ClientWorkProcess

Embora a lógica do TF acima faça o treinamento local com o recorte, ela ainda precisará ser envolvida em código do TFF para criar o bloco básico necessário.

Especificamente, os 4 blocos básicos são representados como um [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess). Isso significa que todos os 4 blocos têm funções `initialize` e `next` para instanciar e executar a computação.

Dessa forma, cada bloco básico pode monitorar seu próprio **estado** (armazenado no servidor) conforme necessário para realizar as operações. Apesar de não ser incluído neste tutorial, isso pode ser usado para, por exemplo, monitorar o número de iterações realizadas ou acompanhar os estados do otimizador.

A lógica do TF de trabalho do cliente deve ser geralmente envolvida como um [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess), que codifica os tipos esperados que entram e saem do treinamento local do cliente. Ele pode ser parametrizado por um modelo e otimizador, conforme abaixo.

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.models.VariableModel],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.models.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# Composição de um algoritmo de aprendizado

Vamos colocar o trabalho do cliente acima em um algoritmo completo. Primeiro, configure os dados e o modelo.

## Prepare os dados de entrada

Carregue e pré-processe o dataset EMNIST incluído no TFF. Para mais detalhes, veja o tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb).

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Para alimentar o dataset no modelo, os dados são achatados e convertidos em tuplas de formato `(flattened_image_vector, label)`.

Selecione um número pequeno de clientes e aplique o pré-processamento acima aos datasets deles.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Prepare o modelo

É usado o mesmo modelo do tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb). Esse modelo (implementado pelo `tf.keras`) tem uma única camada oculta, seguida por uma camada softmax. Para usá-lo no TFF, o modelo do Keras é envolvido como um [`tff.learning.models.VariableModel`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model). Assim, é possível realizar o [passo para frente](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) do modelo no TFF e [extrair as saídas do modelo](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics). Para mais detalhes, veja também o tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb).

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Prepare os otimizadores

Assim como na [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg), há dois otimizadores aqui: um do cliente e outro do servidor. Para simplificar, os otimizadores serão SGD com diferentes taxas de aprendizado.

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## Defina os blocos básicos

Agora que os blocos básicos, os dados, o modelo e os otimizadores do trabalho do cliente foram configurados, falta criar os blocos básicos para o distribuidor, o agregador e o finalizador. Isso pode ser realizado ao usar alguns padrões disponíveis no TFF e que são usados pela FedAvg.

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.models.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## Composição dos blocos básicos

Por fim, você pode usar um **composer** integrado no TFF para reunir os blocos básicos. É um composer relativamente simples, que recebe os 4 blocos básicos acima e liga os tipos.

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# Execute o algoritmo

Agora que o algoritmo está pronto, vamos executá-lo. Primeiro, **inicialize** o algoritmo. O **estado** desse algoritmo tem um componente para cada bloco básico, além de um para os *pesos do modelo global*.

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

Conforme esperado, o trabalho do cliente tem um estado vazio (lembre-se do código do trabalho do cliente acima). No entanto, outros blocos básicos podem ter um estado que não seja vazio. Por exemplo, o finalizador monitora quantas iterações ocorreram. Como `next` ainda não foi executado, o estado é `0`.

In [ ]:
state.finalizer

[0]

Agora realize uma rodada de treinamento.

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

A saída disso (`tff.learning.templates.LearningProcessOutput`) tem ambas as saídas `.state` e `.metrics`. Vamos conferir as duas.

In [ ]:
learning_process_output.state.finalizer

[1]

Claramente, o estado do finalizador foi incrementado por um, já que foi realizada uma rodada de `.next`.

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

Embora as métricas estejam vazias, para algoritmos mais complexos e práticos, elas geralmente estarão cheias de informações úteis.

# Conclusão

Ao usar o framework de blocos básicos/composers acima, você pode criar algoritmos de aprendizado completamente novos, sem precisar fazer tudo novamente do zero. No entanto, esse é apenas o ponto de partida. Esse framework facilita bastante a expressão de algoritmos como modificações simples da FedAvg. Para mais algoritmos, veja [`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms), que contém algoritmos como [FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) e [FedAvg com a programação da taxa de aprendizado do cliente](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule). Essas APIs podem até ajudar nas implementações de algoritmos totalmente novos, como [clustering k-means federado](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans).